# 🔮 What-If Engine - Analisi Scenari

## Obiettivo

Questo notebook implementa un **"What-If Engine"** per esplorare come modifiche comportamentali influenzano il rischio di burnout.

### Caso d'Uso
Un dipendente con alto rischio di burnout vuole sapere:
> "Se dormo 1 ora in più e riduco 2 ore di screen time, il mio rischio diminuisce?"

### Come Funziona
1. Prendiamo un esempio dal dataset (es. un caso di alto burnout)
2. Applichiamo modifiche ("deltas") a specifiche features
3. Confrontiamo le probabilità predette prima/dopo

### Applicazioni Pratiche
- **HR Analytics**: identificare interventi personalizzati per dipendenti a rischio
- **Self-monitoring**: app di wellness che suggerisce cambiamenti comportamentali
- **Policy making**: valutare impatto di politiche aziendali (es. riduzione orario)

### Limitazioni
- Il modello predice **correlazioni**, non **causalità**
- Gli interventi reali potrebbero avere effetti diversi
- Dataset sintetico: validare su dati reali prima del deployment

In [ ]:
# =============================================================================
# SETUP E CARICAMENTO MODELLO
# =============================================================================
# Carichiamo il modello MLP allenato per fare predizioni what-if

import numpy as np
import pandas as pd
import torch
from torch import nn
from pathlib import Path
from sklearn.model_selection import train_test_split
import joblib

# Paths
DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../models/saved')

# =============================================================================
# CARICAMENTO DATI
# =============================================================================
# Usiamo lo stesso dataset del training per avere features consistenti

df = pd.read_parquet(DATA_DIR / 'tabular_ml_ready.parquet')
feature_cols = [c for c in df.columns if c not in {'burnout_level', 'burnout_score'}]
X = df[feature_cols].values.astype(np.float32)
y = df['burnout_level'].values.astype(np.int64)

# Split identico al training (stesso random_state!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convertiamo in DataFrame per avere nomi colonne
X_train = pd.DataFrame(X_train, columns=feature_cols)
X_test = pd.DataFrame(X_test, columns=feature_cols)

# =============================================================================
# CARICAMENTO MODELLO MLP
# =============================================================================
# Definiamo la stessa architettura usata nel training

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    """Stessa architettura di 03_deep_learning_mlp.ipynb"""
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
        )
    def forward(self, x):
        return self.net(x)

# Carica il modello salvato
model_path = MODEL_DIR / 'mlp_classifier.pt'
if model_path.exists():
    checkpoint = torch.load(model_path, map_location=DEVICE, weights_only=False)
    mlp = MLP(len(feature_cols), 3).to(DEVICE)
    mlp.load_state_dict(checkpoint['model_state'])
    mlp.eval()  # Modalità inferenza
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model not found at {model_path}. Run train_mlp.py first.")

# Carica scaler (opzionale, per de-normalizzare le features)
scaler_path = DATA_DIR / 'feature_scaler.joblib'
scaler = joblib.load(scaler_path) if scaler_path.exists() else None
feature_names = feature_cols

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/X_train.joblib'

In [ ]:
# =============================================================================
# WHAT-IF FUNCTION
# =============================================================================
# Funzione principale per analisi scenari

def what_if_scenario(model, x_row: pd.Series, deltas: dict, device=DEVICE):
    """
    Analizza come le modifiche alle features cambiano le predizioni.
    
    Args:
        model: Modello PyTorch allenato
        x_row: Una riga di features (pd.Series con nomi colonne)
        deltas: Dict {nome_feature: variazione}
                Es: {"sleep_hours_mean": +1.0} = +1 ora di sonno
        device: Torch device
    
    Returns:
        dict con:
        - x_base: features originali
        - x_new: features modificate
        - base_proba: probabilità per classe (originali)
        - new_proba: probabilità per classe (dopo modifica)
        - proba_change: differenza new - base
    """
    x_base = x_row.copy()
    x_new = x_row.copy()
    
    # Applica le modifiche
    for feat, delta in deltas.items():
        if feat in x_new.index:
            x_new[feat] = x_new[feat] + delta
        else:
            print(f"⚠️ Warning: feature '{feat}' not found in data")

    # Predizioni
    model.eval()
    with torch.no_grad():
        # Converti in tensori
        x_base_tensor = torch.from_numpy(
            x_base.values.astype(np.float32)
        ).unsqueeze(0).to(device)
        x_new_tensor = torch.from_numpy(
            x_new.values.astype(np.float32)
        ).unsqueeze(0).to(device)
        
        # Forward pass
        base_logits = model(x_base_tensor)
        new_logits = model(x_new_tensor)
        
        # Converti logits in probabilità
        base_proba = torch.softmax(base_logits, dim=1).cpu().numpy()[0]
        new_proba = torch.softmax(new_logits, dim=1).cpu().numpy()[0]

    return {
        "x_base": x_base,
        "x_new": x_new,
        "base_proba": base_proba,           # [P(low), P(medium), P(high)]
        "new_proba": new_proba,
        "proba_change": new_proba - base_proba,  # Differenza
    }

## 🧪 Esempio: Scenario di Intervento

Prendiamo un caso di **alto burnout** dal test set e simuliamo un intervento:
- +1 ora di sonno (`sleep_hours_mean`)
- -2 ore di screen time (`screen_time_hours_mean`)
- -1 ora di lavoro (`work_hours_mean`)

Questo simula una settimana con migliore work-life balance.

In [ ]:
# =============================================================================
# ESECUZIONE WHAT-IF ANALYSIS
# =============================================================================

# Troviamo un caso di alto burnout (classe 2) per l'analisi
mask_high = (y_test == 2)
high_indices = np.where(mask_high)[0]

if len(high_indices) > 0:
    # Prendiamo il primo caso di alto burnout
    idx = high_indices[0]
    x_example = X_test.iloc[idx]
    
    # Definiamo lo scenario: miglioriamo sonno, riduciamo screen time e lavoro
    deltas = {
        "sleep_hours_mean": +1.0,       # +1 ora di sonno medio
        "screen_time_hours_mean": -2.0,  # -2 ore di screen time medio
        "work_hours_mean": -1.0,         # -1 ora di lavoro medio
    }
    
    # Eseguiamo l'analisi
    result = what_if_scenario(mlp, x_example, deltas)
    
    # =============================================================================
    # VISUALIZZAZIONE RISULTATI
    # =============================================================================
    print("=" * 50)
    print("🔮 WHAT-IF ANALYSIS RESULTS")
    print("=" * 50)
    
    print("\n📊 Scenario:")
    for feat, delta in deltas.items():
        sign = "+" if delta > 0 else ""
        print(f"   {feat}: {sign}{delta}")
    
    print(f"\n📈 Original prediction (Low, Medium, High):")
    print(f"   {result['base_proba'].round(3)}")
    
    print(f"\n📉 Modified prediction (Low, Medium, High):")
    print(f"   {result['new_proba'].round(3)}")
    
    print(f"\n🔄 Probability change:")
    print(f"   {result['proba_change'].round(3)}")
    
    # Interpretazione automatica
    print("\n💡 Interpretation:")
    if result['proba_change'][2] < 0:
        reduction = abs(result['proba_change'][2] * 100)
        print(f"   ✅ High burnout risk DECREASED by {reduction:.1f}%")
    else:
        increase = result['proba_change'][2] * 100
        print(f"   ⚠️ High burnout risk INCREASED by {increase:.1f}%")
    
    if result['proba_change'][0] > 0:
        improvement = result['proba_change'][0] * 100
        print(f"   ✅ Low burnout probability INCREASED by {improvement:.1f}%")

else:
    print("❌ No high burnout cases found in test set")

## 📝 Conclusioni

### Risultati dell'Analisi
L'esempio mostra come piccoli cambiamenti comportamentali possano ridurre significativamente il rischio di burnout:
- **+1 ora sonno**: migliora recupero fisico e mentale
- **-2 ore screen time**: riduce affaticamento visivo e stimolazione digitale
- **-1 ora lavoro**: migliora work-life balance

### Prossimi Sviluppi
1. **UI interattiva**: slider per modificare features in tempo reale
2. **Batch analysis**: analizzare intero team per interventi mirati
3. **Modello causale**: usare causal inference per validare interventi
4. **Integrazione app**: collegare a app wellness per raccomandazioni personalizzate

### Limitazioni
- Le predizioni sono basate su correlazioni, non causalità
- Il dataset è sintetico - validare su dati reali
- Gli effetti reali dipendono da molti fattori non modellati